In [ ]:
!pip install -q llama-index-core
!pip install -q llama-index-embeddings-mistralai
!pip install -q llama-index-llms-mistralai
!pip install -q llama-index-readers-file
!pip install mistralai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.5 MB/s eta 0:00:00


In [ ]:

import nest_asyncio

nest_asyncio.apply()

import os

In [ ]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings

In [ ]:
from google.colab import userdata

os.environ["MISTRAL_API_KEY"] = userdata.get('MISTRAL_API_KEY')

In [ ]:
llm = MistralAI(model="open-mixtral-8x22b", temperature=0.1)
embed_model = MistralAIEmbedding(model_name="mistral-embed")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data
We will use Uber-2021 and Lyft-2021 10K SEC filings.

In [ ]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O './uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O './lyft_2021.pdf'

--2024-04-17 19:58:36--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘./uber_2021.pdf’

./uber_2021.pdf     100%[===================>]   1.79M  --.-KB/s    in 0.04s   

2024-04-17 19:58:37 (47.2 MB/s) - ‘./uber_2021.pdf’ saved [1880483/1880483]

--2024-04-17 19:58:37--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP requ

### Load the Data


In [ ]:
from llama_index.core import SimpleDirectoryReader

uber_docs = SimpleDirectoryReader(input_files=["./uber_2021.pdf"]).load_data()
lyft_docs = SimpleDirectoryReader(input_files=["./lyft_2021.pdf"]).load_data()

In [ ]:
# uber_docs

### Build RAG on uber and lyft docs

Separte Vector Stores for both companies


In [ ]:
from llama_index.core import VectorStoreIndex

uber_index = VectorStoreIndex.from_documents(uber_docs)
uber_query_engine = uber_index.as_query_engine(similarity_top_k=5)

lyft_index = VectorStoreIndex.from_documents(lyft_docs)
lyft_query_engine = lyft_index.as_query_engine(similarity_top_k=5)

#### Ask questions

In [ ]:
response = uber_query_engine.query("What is the revenue of uber in 2021?")
print(response)

Uber's revenue in 2021 was $17,455 million.


In [ ]:
# response

In [ ]:
response = lyft_query_engine.query("What are lyft investments in 2021?")
print(response)

In 2021, Lyft invested in various areas to improve its services and expand its offerings. These investments include:

1. Expansion of Light Vehicles and Lyft Autonomous: Lyft continued to invest in the expansion of its network of Light Vehicles and Lyft Autonomous, focusing on the deployment and scaling of third-party self-driving technology on the Lyft network.

2. Improving Driver Experience: Lyft invested in its Express Drive program, which provides drivers with access to rental cars for ridesharing. The company also invested in its Driver Centers, Mobile Services, and related partnerships to offer drivers affordable and convenient vehicle maintenance services.

3. Marketplace Technology: Lyft invested in its proprietary technology to deliver a convenient and high-quality experience to drivers and riders. These investments included mapping, routing, payments, in-app navigation, matching technologies, and data science.

4. M&A and Strategic Partnerships: In 2021, Lyft continued to se

#### FunctionCallingAgent with RAG QueryEngineTools

Here we use Fuction Calling capabilities of the model. Automatically routing the query to appropriate VectorStore



In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner



In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_query_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_query_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021",
        ),
    ),
]


In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("What is the revenue of uber in 2021.")

Added user message to memory: What is the revenue of uber in 2021.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "revenue"}
=== Function Output ===
Uber's revenue is primarily derived from fees paid by Mobility Drivers for using their platforms and related services to facilitate and complete Mobility services. Additionally, revenue is generated from fees paid by end-users for connection services obtained via the platform. Uber's revenue also includes immaterial revenue streams such as financial partnerships products and Vehicle Solutions.

Uber's Delivery revenue is derived from Merchants' and Couriers' use of the Delivery platform and related services to facilitate and complete Delivery transactions. In certain markets where Uber is responsible for delivery services, delivery fees charged to end-users are also included in revenue. Delivery revenue also includes advertising revenue from sponsored listing fees paid by merchants and brands in exchange for adver

In [ ]:
print(response)

assistant: According to the information provided, Uber's total revenue for the year 2021 was $17,455 million. This revenue is derived from various sources such as Mobility, Delivery, Freight, and All Other revenue streams. The Mobility revenue was $6,953 million, Delivery revenue was $8,362 million, Freight revenue was $2,132 million, and All Other revenue was $8 million.


In [ ]:
response = agent.chat("What are lyft investments in 2021?")

Added user message to memory: What are lyft investments in 2021?
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "investments"}
=== Function Output ===
The company's investments include cash and cash equivalents, short-term investments, and restricted investments. Cash equivalents consist of certificates of deposits, commercial paper, and corporate bonds with an original maturity of 90 days or less. Short-term investments are comprised of commercial paper, certificates of deposit, and corporate bonds that mature in twelve months or less. Restricted investments are held in trust accounts at third-party financial institutions and include debt security investments in commercial paper, certificates of deposit, corporate bonds, and U.S. government securities. The company also has investments in non-marketable equity securities, which are measured at cost with remeasurements to fair value only upon the occurrence of observable transactions for identical or similar in

In [ ]:
print(response)

assistant: According to the information provided, Lyft's investments in 2021 include cash and cash equivalents, short-term investments, and restricted investments. Cash equivalents consist of certificates of deposits, commercial paper, and corporate bonds with an original maturity of 90 days or less. Short-term investments are comprised of commercial paper, certificates of deposit, and corporate bonds that mature in twelve months or less. Restricted investments are held in trust accounts at third-party financial institutions and include debt security investments in commercial paper, certificates of deposit, corporate bonds, and U.S. government securities. The company also has investments in non-marketable equity securities, which are measured at cost with remeasurements to fair value only upon the occurrence of observable transactions for identical or similar investments of the same issuer or impairment.


### Agents and Tools usage

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import (
    FunctionCallingAgentWorker,
    AgentRunner,
    ReActAgent,
)

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

#### With Function Calling.

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool, subtract_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("What is (26 * 2) + 2024?")
print(response)

Added user message to memory: What is (26 * 2) + 2024?
=== Calling Function ===
Calling function: multiply with args: {"a": 26, "b": 2}
=== Function Output ===
52
=== LLM Response ===
The result of 26 * 2 is 52.
Now, let's add 2024 to this result.

Next, I will call the add function with arguments a=52 and b=2024.

Here is the code:

[{"name": "add", "arguments": {"a": 52, "b": 2024}}]
assistant: The result of 26 * 2 is 52.
Now, let's add 2024 to this result.

Next, I will call the add function with arguments a=52 and b=2024.

Here is the code:

[{"name": "add", "arguments": {"a": 52, "b": 2024}}]
